In [2]:
'''
@name sudoku-solver
@brief 数独の問題を解く
@note
使用方法

problem = [
 [0, 9, 0, 0, 0, 0, 0, 3, 0],
 [7, 0, 0, 1, 0, 3, 0, 0, 8],
 [0, 0, 8, 0, 7, 0, 1, 0, 0],
 [0, 0, 9, 0, 0, 0, 5, 0, 0],
 [8, 0, 0, 2, 0, 9, 0, 0, 4],
 [0, 0, 5, 0, 0, 0, 6, 0, 0],
 [0, 0, 1, 0, 8, 0, 2, 0, 0],
 [6, 0, 0, 5, 0, 2, 0, 0, 1],
 [0, 3, 0, 0, 0, 0, 0, 4, 0]]

sudoku_solver(problem)
'''
import numpy as np
import copy

def sudoku_solver(problem, debug=False):
    prob = np.array(problem).astype(np.int32)

    if prob.shape != (9,9):
        print('問題が 9 x 9 になっていません')
        return
    
    if prob.min()<0 or 9<prob.max():
        print('値は0から9の整数にしてください')
        return
    
    board = prob.copy()
    print('問題')
    print(board)
    
    # board_numsテーブルの初期化 セル毎に可能性のある数字の集合を示す
    board_nums = np.full((9,9),set())
    for r in range(0,9):
        for c in range(0,9):
            board_nums[r][c]=set(range(1,10))
    
    for r in range(0,9):
        for c in range(0,9):
            if board[r][c]>0:
                decide_val(board_nums, board, r, c, board[r][c])
    
    if debug:
        print(board_nums)
    
    # create group
    group = []
    for r in range(0,9):
        group.append([(r,c) for c in range(0,9)])
    for c in range(0,9):
        group.append([(r,c) for r in range(0,9)])
    for mr in range(0,9,3):
        for mc in range(0,9,3):
            group.append([(mr+i,mc+j) for i in range(0, 3) for j in range(0, 3)])          
    
    # count_by_logical_level
    cntbl = {'L1':0, 'L2':0, 'L3_2':0, 'L3_3':0, 'L3_x':0, 'L4':0}

    ret, ret_board = iterate_process(group, board_nums, board, cntbl, debug=debug)
        
    # print('処理終了 回答')
    print(ret_board)
    print('ロジック別カウント')
    print('  L1: セルの候補値が1個')
    print('  L2: 行,列,ブロック別に判定時に候補数字が1個のセルのみ')
    print('  L3_2: 行,列,ブロック別に判定時に候補値集合が2個のセルで同一かつ集合サイズが2')
    print('  L3_3: 行,列,ブロック別に判定時に候補値集合が3個のセルで同一かつ集合サイズが3')
    print('  L3_x: 同上,　ただし個数は4以上')
    print('  L4: 数字の仮置発生')

    print(cntbl)
        
def iterate_process(group, board_nums, board, cntbl, debug=False):
    for i in range(1, 100):
        if debug:
            print("Loop :",i)

        before_board_nums = board_nums.copy()
        
        if detect_unique_num(group, board_nums, board):
            cntbl['L1'] += 1
            continue
        
        if decide_multi_cells_vals(group, board_nums, board, cntbl, debug=debug):
            continue
            
        if detect_single_num(group, board_nums, board):
            cntbl['L2'] += 1
            continue

        if np.count_nonzero(board_nums == set() )==81:
            if ck_result(group, board):
                print('** 成功 **')
                return True, copy.deepcopy(board)
            else:
                # print('** Error 問題がおかしい可能性もある **')
                return False, copy.deepcopy(board)

        if np.array_equal(before_board_nums, board_nums):
            cntbl['L4'] += 1
            # 最も少ない候補値を持つセルに対して、候補値を1つづつ試してゆく
            n_min,rc, nums = 9, None, None
            for r in range(0,9):
                for c in range(0,9):
                    if len(board_nums[r][c])>1 and len(board_nums[r][c])<n_min:
                        n_min = len(board_nums[r][c])
                        rc = [r,c]
                        nums = tuple(board_nums[r][c])
            if debug:
                print(f'仮 {tuple(rc)} vals:{nums}')
            
            for val in nums:
                if debug:
                    print(f'仮 val={val}')
                    print(board)
                    print(board_nums)
                bk_board = copy.deepcopy(board)
                bk_board_nums = copy.deepcopy(board_nums)
                
                decide_val(bk_board_nums, bk_board, rc[0], rc[1], val)
                
                # recursion
                ret, ret_board= iterate_process(group, bk_board_nums, bk_board, cntbl, debug=debug)
                if ret:
                    if debug:
                        print('-- 成功しました --')
                    return True, copy.deepcopy(ret_board)

            if debug:
                print('-- 失敗しました --')
            return False, copy.deepcopy(board)


def decide_multi_cells_vals(group, board_nums, board, cntbl, debug=False):
# 3✖️3マトリックス,行, 列ごとに、複数セルのセル候補値が同一かつそのセル数と候補値数が一致するなら、他のセルの候補値ではない

    for g in group:
        wk = {}
        for rc in g:
            r,c = rc[0], rc[1]
            if len(board_nums[r][c])>1:
                wk[tuple(board_nums[r][c])] = []
                
        for rc in g:
            r,c = rc[0], rc[1]
            if len(board_nums[r][c])>1:
                wk[tuple(board_nums[r][c])].append((r,c))

        for k in wk.keys():
            if len(k)==len(wk[k]) and len(k)>1:
                change_flag = False
                for rc in g:
                    r,c = rc[0], rc[1]
                    if set(k) != board_nums[r][c]:
                        before = board_nums[r][c].copy()
                        board_nums[r][c]-= set(k)
                        if before != board_nums[r][c]:
                            change_flag = True
                            if debug:
                                print(f'decide_multi_cells_vals ({r},{c}) {before} -->{board_nums[r][c]}')

                if change_flag:
                    if len(k)==2:
                        cntbl['L3_2'] += 1
                    elif len(k)==3:
                        cntbl['L3_3'] += 1
                    else:
                        cntbl['L3_x'] += 1
                    return True
    return False

# 行, 列, 3✖️3マトリックスごとの候補値に、1回しか現れない数字があれば、その値に決定
def detect_single_num(group, board_nums, board):    
    for g in group:
        cnts = {x:[] for x in range(1,10)}           
        for rc in g:
            for n in board_nums[rc[0]][rc[1]]:
                cnts[n].append(rc)

        for n in range(1,10):
            if len(cnts[n])==1:
                rs = cnts[n][0][0]
                cs = cnts[n][0][1]
                decide_val(board_nums, board, rs, cs, n)
                return True
    return False

def detect_unique_num(group, board_nums, board):
    for g in group:
        for rc in g:
            if len(board_nums[rc[0]][rc[1]])==1:  # 候補が1つしかない
                val = list(board_nums[rc[0]][rc[1]])[0]
                decide_val(board_nums, board, rc[0], rc[1], val)
                return True
    return False

def decide_val(board_nums, board, r, c, val):
# board_numsの更新処理
# 行, 列, 3✖️3マトリックスごとに、すでにセル値が定まっている場合に、他のセルの候補値集合からその値を除く
     
    board[r][c] = val
    board_nums[r][c] = set()
        
    # 行
    for cs in range(0,9):
        if cs != c:
            board_nums[r][cs].discard(val)
        
    # 列
    for rs in range(0,9):
        if rs != r:
            board_nums[rs][c].discard(val)
    
    # 3 x 3 matrix
    rm = r - r%3
    cm = c - c%3
    for i in range(0,3):
        for j in range(0,3):
            rs = rm + i
            cs = cm + j
            if rs != r and cs != c:
                board_nums[rs][cs].discard(val)

def ck_result(group, board):
    # sudokuのルールに沿ったチェック
    for g in group:
        lis = [ board[rc[0], rc[1]] for rc in g if board[rc[0], rc[1]]>0 ]
        if not(len(lis)==9 and len(set(lis))==9):
            return False
    return True


In [37]:
# Minimum Sudoku 2919
problem = [
 [0, 0, 0, 0, 1, 0, 6, 0, 0],
 [3, 0, 0, 0, 0, 0, 0, 2, 0],
 [7, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 7, 0, 2, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 8, 0, 0],
 [5, 0, 0, 3, 0, 0, 0, 0, 0],
 [0, 0, 0, 2, 0, 0, 0, 3, 5],
 [4, 0, 0, 0, 0, 0, 0, 0, 7],
 [0, 6, 0, 0, 0, 0, 0, 0, 0]]

# sudoku_solver(problem, debug=True)
sudoku_solver(problem)

問題
[[0 0 0 0 1 0 6 0 0]
 [3 0 0 0 0 0 0 2 0]
 [7 0 0 0 0 0 0 0 0]
 [0 0 0 7 0 2 0 0 0]
 [0 1 0 0 0 0 8 0 0]
 [5 0 0 3 0 0 0 0 0]
 [0 0 0 2 0 0 0 3 5]
 [4 0 0 0 0 0 0 0 7]
 [0 6 0 0 0 0 0 0 0]]
** 成功 **
[[8 2 4 5 1 7 6 9 3]
 [3 5 1 6 4 9 7 2 8]
 [7 9 6 8 2 3 5 1 4]
 [9 4 8 7 6 2 3 5 1]
 [6 1 3 9 5 4 8 7 2]
 [5 7 2 3 8 1 9 4 6]
 [1 8 9 2 7 6 4 3 5]
 [4 3 5 1 9 8 2 6 7]
 [2 6 7 4 3 5 1 8 9]]
ロジック別カウント
  L1: セルの候補値が1個
  L2: 行,列,ブロック別に判定時に候補数字が1個のセルのみ
  L3_2: 行,列,ブロック別に判定時に候補値集合が2個のセルで同一かつ集合サイズが2
  L3_3: 行,列,ブロック別に判定時に候補値集合が3個のセルで同一かつ集合サイズが3
  L3_x: 同上,　ただし個数は4以上
  L4: 数字の仮置発生
{'L1': 133, 'L2': 39, 'L3_2': 22, 'L3_3': 8, 'L3_x': 2, 'L4': 3}


In [4]:
# The hardest sudokus  Rating 99529
problem = [
 [1, 2, 0, 4, 0, 0, 3, 0, 0],
 [3, 0, 0, 0, 1, 0, 0, 5, 0],
 [0, 0, 6, 0, 0, 0, 1, 0, 0],
 [7, 0, 0, 0, 9, 0, 0, 0, 0],
 [0, 4, 0, 6, 0, 3, 0, 0, 0],
 [0, 0, 3, 0, 0, 2, 0, 0, 0],
 [5, 0, 0, 0, 8, 0, 7, 0, 0],
 [0, 0, 7, 0, 0, 0, 0, 0, 5],
 [0, 0, 0, 0, 0, 0, 0, 9, 8]]

# sudoku_solver(problem, debug=True)
sudoku_solver(problem)

問題
[[1 2 0 4 0 0 3 0 0]
 [3 0 0 0 1 0 0 5 0]
 [0 0 6 0 0 0 1 0 0]
 [7 0 0 0 9 0 0 0 0]
 [0 4 0 6 0 3 0 0 0]
 [0 0 3 0 0 2 0 0 0]
 [5 0 0 0 8 0 7 0 0]
 [0 0 7 0 0 0 0 0 5]
 [0 0 0 0 0 0 0 9 8]]
** 成功 **
[[1 2 8 4 6 5 3 7 9]
 [3 7 4 2 1 9 8 5 6]
 [9 5 6 8 3 7 1 4 2]
 [7 6 5 1 9 8 4 2 3]
 [2 4 9 6 7 3 5 8 1]
 [8 1 3 5 4 2 9 6 7]
 [5 9 2 3 8 6 7 1 4]
 [4 8 7 9 2 1 6 3 5]
 [6 3 1 7 5 4 2 9 8]]
ロジック別カウント
  L1: セルの候補値が1個
  L2: 行,列,ブロック別に判定時に候補数字が1個のセルのみ
  L3_2: 行,列,ブロック別に判定時に候補値集合が2個のセルで同一かつ集合サイズが2
  L3_3: 行,列,ブロック別に判定時に候補値集合が3個のセルで同一かつ集合サイズが3
  L3_x: 同上,　ただし個数は4以上
  L4: 数字の仮置発生
{'L1': 4936, 'L2': 316, 'L3_2': 499, 'L3_3': 11, 'L3_x': 0, 'L4': 198}
